## DATASET CREATION

Unfortunately, the dataset I will work with does not come as one already done dataset - as it is mostly usual in Data Science projects. <br /> 
To get the sheer experience of how a normal Data Science job looks like I also wanted to dive into this issue. By experience I can tell that this takes a shit ton of time - like almost 80% - of preparing the data. <br /> 
Since the data usually shares the same countries and lists mostly the same years of the recording I used this as a primary key where I connect the datapoints with each other. <br /> 
However, the  countries are listed in rows along with the year of the recording - I want to have a final dataset that looks like follows: <br /> 

|Country | Afghanistan | Albania | ... | Zimbabwe | 
| ----- | ----------  | ------ | ----- | ------- | 
|Alcohol consumption [l] | 0.2 | 2.4 | ... | 0.01 |
|Human Develpment Index (HDI) | 0.1 | 0.15 | ... | 0.1 | 
|... | ... | ... | ... | ... |
|Healthcare Expenditure [$] | 13.322 | 15.211 | ... | 1.039 |

Thus, I have to transpose each of the countries and record each of the years as seperate entry in the dataset. <br /> 

All the data is [publicly available](https://ourworldindata.org), and this source is trusted by many notorious companies such as Vox, The Ney York times and even the top universities of this world like MIT, Oxford, Stanford. <br /> 
Hence, I assume that this data is rather based on actual recording from the respective country. <br /> 
Even the United Nation published their records in this page and I bet that these folks do some amazing work, which we can trust. <br />


But now let's not waste too much with the explaination part and go straight into how I merged the datafiles to one huge on. <br /> 

In [1]:
import os 
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

Do the preprocessing necessities with renaming the columns and dropping the ones we are not interested in, i.e. continents aggregated stuff, some islands and countries I have never heard of, etc. <br /> 
After doing that, we save the new csv file again. 

In [2]:
def rename_Countries_drop_Unnecessary(df, name): 
    print("Before {} shape: {}".format(name, df.shape))
    # Input is only the dataframe with the country names in the column 'Entity'
    right_names = []
    for data in df['Entity']:
        ## RENAMING COUNTRIES FOR DATA CONSISTENCY
        if 'Hong Kong' in data:
            data = 'Hong Kong'
        if 'Taiwan' in data: 
            data = 'Taiwan'
        if 'Macao' in data: 
            data = 'Macao'
        if 'Ethiopia' in data: 
            data = 'Ethiopia'
        if 'Sudan' in data: 
            data = 'Sudan'        
        if 'Czechia' in data: 
            data = 'Czech Republic'
        if 'Syria' in data: 
            data = 'Syria'
        if 'Russ' in data: 
            data = 'Russia'
        if "Ivoire" in data:
            data = "Cote d'Ivoire"

        # America 
        if 'US' in data: 
            data = 'United States'
        if 'USA' in data: 
            data = 'United States'
        if 'U.S.A.' in data: 
            data = 'United States'
        if 'U.S.A' in data: 
            data = 'United States'
        if 'United States of' in data: # gets United States of America
            data = 'United States'
        if data == 'America': 
            data = 'United States'
        right_names.append(data) 
        
    ## Replace the names with the consistent names of them
    right_names = pd.Series(right_names)
    df['Entity'] = right_names

    ## DELETE THE ENTRIES WHICH ARE NOT IN OUR MASTER COUNTRY LIST
    countries = list(set(df["Entity"]))
    for country in countries: 
        if country not in interested_countries:
            idx = list(df['Entity']).index(country)
            endCountry = idx + list(df['Entity']).count(country)
            ranges = np.arange( idx , endCountry )  
            df.drop(df.index[ranges], inplace = True)
    
    ## SAVE THE FILE REDUCED AND CHANGED NAME AGAIN
    print("After: {}".format(df.shape))
    df.to_csv(os.path.join(datapath, name))

Define the countries we want to have in our Masterlist and thus in the dataset.

In [ ]:
interested_countries = ['Sweden', 'Norway', 'Finland', 'Iceland', 'Germany', 'Netherlands', 'Belgium', 'Luxembourg',
                        'England', 'Scotland', 'Wales', 'Ireland', 'United Kingdom', 'Switzerland', 'Austria', 'France'
                        'Italy', 'Spain', 'Portugal', 'Morocco', 'Tunisia', 'Egypt', 'Liechtenstein', 'Cyprus', 'Vatican',
                        'Kosovo', 'Serbia', 'Georgia', 'Greenland', 'Antigua and Barbuda', 'Hungary', 'Monaco', 'Israel',
                        'Albania', 'Iraq', 'Iran', 'Syria', 'Turkey', 'Palestine', 'Montenegro', 'Latvia', 'Jordan',
                        'Croatia', 'New Zealand', 'Eritrea', 'Libya', 'Belarus', 'Slovenia', 'Greece', 'Lithuania',
                        'Liberia', 'Slovakia', 'Estonia', 'Poland', 'Czech Republic', 'Armenia', 'Denmark', 'Bulgaria',
                        
                        'Russia', 'United States', 'Canada', 'Qatar', 'Kuwait', 'Mexico', 'South Africa', 'Fiji', 'Oman',
                        'Japan', 'United Arab Emirates', 'South Korea', 'Macao', 'Hong Kong', 'China', 'Thailand', 'Belize',
                        'Taiwan', 'Vietnam', 'Malaysia', 'Indonesia', 'India', 'Philippines', 'Australia', 'Laos', 'Bhutan',
                        
                        'Kyrgyzstan', 'Kazakhstan', 'Uzbekistan', 'Turkmenistan', 'Tajikistan', 'Pakistan', 'Afghanistan',
                        'Argentina', 'Brazil', 'Chile', 'Venezuela', 'Peru', 'Colombia', 'Guyana', 'Mauritius', 'Barbados', 
                        'Cuba', 'Panama', 'Bahamas', 'Puerto Rico', 'Costa Rica', 'Solomon Islands',  'Marshall Islands',
                        'Ecuador', 'Benin', 'Seychelles', 'Bolivia', 'Madagascar',  'Mauritania', 'Bosnia and Herzegovina', 
                        'Jamaica', 'Lebanon', 'Senegal', 'Malta', 'French Polynesia', 'Bahrain', 'Burundi', 'Swaziland',
                        'Tanzania', 'Central African Republic', 'Malawi', 'Djibouti', 'Mozambique', 'Macedonia', 'Sierra Leone',
                        'Democratic Republic of Congo', 'Namibia', 'Algeria', 'Trinidad and Tobago', "Cote d'Ivoire",
                         
                        'Samoa', 'Bermuda', 'Aruba', 'Myanmar', 'Cape Verde', 'Uganda', 'Togo', 'Guinea', 
                        'San Marino', 'Ukraine', 'North Korea', 'Papua New Guinea', 'Haiti', 'Ghana', 'Sudan',
                        'Faeroe Islands', 'Cambodia', 'Somalia',  'Kiribati', 'Tonga', 'Mongolia', 'Rwanda', 'Bangladesh',
                        'Suriname', 'Nauru', 'Zambia', 'Azerbaijan',  'Sri Lanka', 'Nigeria', 'Kenya', 'Comoros', 'Andorra', 
                        'Tuvalu', 'Zimbabwe', 'Yemen', 'Cameroon', 'El Salvador', 'Angola', 'Curacao', 'Nicaragua',
                        'Saudi Arabia', 'Lesotho', 'Moldova', 'Gabon', 'Grenada', 'Mali', 'Romania', 'Guatemala', 'Dominican Republic', 
                        'Honduras', 'Congo',  'Burkina Faso',  'Saint Lucia', 'Cayman Islands', 'Botswana', 'Ethiopia', 
                        'Chad', 'Uruguay', 'Maldives', 'Gibraltar', 'Paraguay', 'Niger', 'Nepal']

In [4]:
# Define the path where we have our data stored and want to have it stored as well.
datapath = os.path.join(os.path.join(os.getcwd(), 'data'), 'Health')
datapath

'C:\\Users\\Lenny\\Documents\\Studium_Robotics (M.Sc.)\\03_Semester 3 - Oslo ERASMUS\\01_Applied Data Analysis and Machine Learning\\Project 3\\data\\Health'

Call each single file in our data directory and process each one according to the rules we set previously (Renaming and Deleting entries). 

In [8]:
dataFileNames = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]
type4Cols = []
type7Cols = []
manualLists = []

# the datasets mostly have the same size of 4 columns and same setup so let's get those first

for file in dataFileNames: 
    try:
        df = pd.read_csv(os.path.join(datapath, str(file) ) , encoding='latin-1')
    except: 
        print("problems with this guy: {}".format(file))
        manualLists.append(file)
    if df.shape[1] == 4: # one type of files (4 columns) 
        type4Cols.append(file) 
        rename_Countries_drop_Unnecessary(df, str(file))
        
    elif df.shape[1] == 7: 
        type7Cols.append(file) 
    else: # manual shit  to do then 
        manualLists.append(file)

problems with this guy: 00_insufficient-activity-adults.xlsx
problems with this guy: 00_mean-systolic-blood-pressure.xlsx
problems with this guy: 00_mean-total-cholesterol.xlsx
Before agricultural-area-per-capita.csv shape: (8993, 4)
After: (7753, 4)
Before alcohol-attributable-fraction-of-mortality.csv shape: (190, 4)
After: (173, 4)
Before annual-healthcare-expenditure-per-capita.csv shape: (4675, 4)
After: (3460, 4)
Before average-height-of-men-for-selected-countries.csv shape: (1250, 4)
After: (1203, 4)
Before beer-consumption-per-person.csv shape: (8952, 4)
After: (8268, 4)
Before cancer-death-rates.csv shape: (6468, 4)
After: (5068, 4)
Before cardiovascular-disease-death-rates.csv shape: (6468, 4)
After: (5068, 4)
Before child-mortality.csv shape: (13512, 4)
After: (12383, 4)
Before co-emissions-per-capita.csv shape: (42844, 4)
After: (39453, 4)
Before consumption-per-smoker-per-day.csv shape: (6204, 4)
After: (5742, 4)
Before dementia-death-rates.csv shape: (6468, 4)
After: (506

C:\Users\Lenny\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Before military-expenditure-as-share-of-gdp.csv shape: (9172, 4)
After: (6667, 4)
Before milk-production-tonnes.csv shape: (11789, 4)
After: (8572, 4)
Before minutes-spent-on-leisure-men.csv shape: (31, 4)
After: (28, 4)
Before minutes-spent-on-leisure-women.csv shape: (31, 4)
After: (28, 4)
Before number-of-nurses.csv shape: (650, 4)
After: (582, 4)
Before per-capita-egg-consumption-kilograms-per-year.csv shape: (10393, 4)
After: (7714, 4)
Before per-capita-milk-consumption.csv shape: (10393, 4)
After: (7714, 4)
Before pisa-test-score-mean-performance-on-the-mathematics-scale.csv shape: (341, 4)
After: (326, 4)
Before pisa-test-score-mean-performance-on-the-science-scale.csv shape: (344, 4)
After: (329, 4)
Before population-density.csv shape: (14600, 4)
After: (10623, 4)
Before prevalence-of-anemia-in-pregnant-women.csv shape: (6093, 4)
After: (4725, 4)
Before prevalence-of-vitamin-a-deficiency-in-pregnant-women.csv shape: (162, 4)
After: (142, 4)
Before respiratory-disease-death-rate

Set up the final dataframe which we are going to use in the Analysis part. <br /> 
Notice that I set it up with 5mio rows, however this is just to ensure that all the data will be safely stored in it. I will delete the empty rows after the dataset is created. <br /> 
So, it's just a placeholder until now and serves the purpose of not running into index errors/ too small row size.

In [10]:
# create dataframe where we want to paste everything inside
# 5.000.000 rows to not run into some problems while adding rows - delete later the other ones
final_df = pd.DataFrame(data = np.zeros( (5000000, len(interested_countries)) ), 
                        index = np.arange(5000000),
                        columns = [ country for country in interested_countries])
# save indices as strings to get meaningful names
final_df.index = final_df.index.map(str)
final_df.shape

(5000000, 196)

Just a Testbench for getting the values I want

In [ ]:
# testing to get values of the specific entries and Co
columns = list(df.columns)
b = df[df['Entity'] == 'France']
c = b[b['Year'] == 1999]#[columns[-1]]
d = b[b['Year'] == 1999][columns[-1]].sum()
print(d)
c
#df.groupby(["Entity", 'Year']).describe()

The Magic happens down here. <br /> 
We loop through every preprocessed list, <br /> 
In each list we loop through every country and further <br /> 
we also iterate over each year in that country. <br /> 
There I use the Year and the name of the file/list to create an index name. In this index name we paste the respective country and its value in it. <br /> 
We do this for all the preprocessed lists, which takes a shit ton of time. <br /> 


In [25]:
final_df[2650:2800]

,Sweden,Norway,Finland,Iceland,Germany,Netherlands,Belgium,Luxembourg,England,Scotland,...,Cayman Islands,Botswana,Ethiopia,Chad,Uruguay,Maldives,Gibraltar,Paraguay,Niger,Nepal
Milk production [t] in 2010,2.902100e+06,1.579920e+06,2.336253e+06,123218.0,2.965287e+07,1.180507e+07,3.075120e+06,297197.0,0.0,0.0,...,0.0,118225.0,4430840.0,271847.0,1820746.0,0.0,0.0,490350.0,967820.0,1584290.0
Milk production [t] in 2011,2.890000e+06,1.548507e+06,2.300711e+06,123997.0,3.035763e+07,1.183543e+07,3.110090e+06,294759.0,0.0,0.0,...,0.0,118505.0,3617505.0,275710.0,2120767.0,0.0,0.0,512000.0,967677.0,1645060.0
Milk production [t] in 2012,2.901000e+06,1.605301e+06,2.296694e+06,125093.0,3.070754e+07,1.189278e+07,3.081213e+06,292800.0,0.0,0.0,...,0.0,119600.0,4085108.0,277015.0,2244480.0,0.0,0.0,515000.0,1012713.0,1711531.0
Milk production [t] in 2013,2.910000e+06,1.598218e+06,2.327800e+06,122900.0,3.136055e+07,1.244342e+07,3.483230e+06,298877.0,0.0,0.0,...,0.0,119625.0,3833029.0,277290.0,2238000.0,0.0,0.0,518000.0,1050629.0,1770212.0
Milk production [t] in 2014,2.973000e+06,1.581887e+06,2.400003e+06,133514.0,3.243221e+07,1.272923e+07,3.701900e+06,319761.0,0.0,0.0,...,0.0,117529.0,3699373.0,291849.0,2231000.0,0.0,0.0,525000.0,1097833.0,1792204.0
Men (15-65) leisure time [min] in 2018,3.137028e+02,3.703389e+02,3.587395e+02,0.0,3.462538e+02,3.006930e+02,3.943693e+02,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Women (15 - 65) leisure time [min] in 2018,2.723751e+02,3.658482e+02,3.036041e+02,0.0,3.157683e+02,2.902997e+02,3.457599e+02,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Number of Nurses in 2016,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2658,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2659,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
%%time
# paste the values into the final_df from each single list
nextListIndex = 2658

try: 
    for lists in type4Cols:
        if type4Cols.index('number-of-nurses.csv') >= type4Cols.index(lists):
            continue
        
        ## Read the file and get the all countries along with their reported years
        print("{} list out of {}, Index: {}, Name: {}".format(type4Cols.index(lists), len(type4Cols), nextListIndex, lists))
        df = pd.read_csv(os.path.join(datapath, str(lists) ) , encoding='latin-1')
        
        # get the col names, unique countries and unique years
        columns = list(df.columns)
        countries = list(set(df["Entity"]))
        years = list(set(df['Year']))
        
        # get a list of all the index/row names
        indexNamesArr = final_df.index.values

        ## groupby countries and then years accordingly
        #df.groupby(["Entity", 'Year'])

        firstListRun = False # flag for renaming the indices

        # loop thru every country in the list
        for country in countries:
            
            ## check if country is in our masterlist
            if country not in interested_countries: 
                # skip this item
                print("\tCountry: {} not in list - but we skip it.".format(country))
                continue
            
            # take a dataframe for one country at a time
            country_df = df[df['Entity'] == country]

            # loop thru every year within that country - assuming the years are in the same order for every country
            for year in years:
                
                # rename the indices only if it is the very first run for the country
                if not firstListRun:
                    indexName = str(columns[-1]) + ' in ' + str(year)
                    indexNamesArr[nextListIndex] = indexName
                    nextListIndex += 1
                    if nextListIndex % 20 == 0: 
                        print("\t\t" + str(indexName))

                ## get the proper value and fill empty ones, if not available, fill it with 0.000
                # note: .sum() is only having one element anyway, just done to get the value as a float not an array
                value = country_df[country_df["Year"] == year][columns[-1]].sum() if not country_df[country_df["Year"] == year][columns[-1]].empty else 0.000
                # get the name of the row
                idxName = str(columns[-1]) + ' in ' + str(year)
                
                #print("Country: {} found in the dataset at spot: {}".format(country, interested_countries.index(country)))
                
                # assign the value in the merged df with the value 
                final_df.iat[list(final_df.index.values).index(idxName), interested_countries.index(country)] = value # .iat[row, col]

            ## get the proper index for the next list to begin with   
            firstListRun = True
            # print progress
            if countries.index(country) % 60 == 0:
                print("\tWorking on country: {} out of {}".format(countries.index(country), len(countries)))
            
except Exception as e:
    print("Next List Index in line: {} of {}, Matrix Size: {}, list: {}, country: {}".format(i, nextListIndex, final_df.shape[0], lists, country))
    print(e)
    sys.exit()

32 list out of 61, Index: 2658, Name: per-capita-egg-consumption-kilograms-per-year.csv
		Egg consumption per capita [kg] in 1962
		Egg consumption per capita [kg] in 1982
		Egg consumption per capita [kg] in 2002
	Working on country: 0 out of 161
	Working on country: 60 out of 161
	Working on country: 120 out of 161
33 list out of 61, Index: 2711, Name: per-capita-milk-consumption.csv
		Milk consumption per capita - Excluding Butter [kg] in 1969
		Milk consumption per capita - Excluding Butter [kg] in 1989
		Milk consumption per capita - Excluding Butter [kg] in 2009
	Working on country: 0 out of 161
	Working on country: 60 out of 161
	Working on country: 120 out of 161
34 list out of 61, Index: 2764, Name: pisa-test-score-mean-performance-on-the-mathematics-scale.csv
	Working on country: 0 out of 74
	Working on country: 60 out of 74
35 list out of 61, Index: 2770, Name: pisa-test-score-mean-performance-on-the-science-scale.csv
	Working on country: 0 out of 74
	Working on country: 60 

In [31]:
final_df[2000:2500]

,Sweden,Norway,Finland,Iceland,Germany,Netherlands,Belgium,Luxembourg,England,Scotland,...,Cayman Islands,Botswana,Ethiopia,Chad,Uruguay,Maldives,Gibraltar,Paraguay,Niger,Nepal
"Maternal Mortality (deaths per 100,000 live births) in 1841",622.60000,0.00000,853.60000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1842",669.50000,0.00000,766.30000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1843",459.90000,0.00000,869.50000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1844",426.00000,0.00000,754.80000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1845",430.30000,0.00000,774.40000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1846",494.50000,0.00000,875.90000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1847",441.60000,0.00000,864.50000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1848",415.50000,0.00000,784.20000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1849",471.90000,0.00000,702.90000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000
"Maternal Mortality (deaths per 100,000 live births) in 1850",467.40000,0.00000,798.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.00000


In [29]:
## Delete all the values which are 0
finale = final_df.copy()
finale.dropna(how = 'all', axis=0)
finale.shape

(5000000, 196)

Finally save our sweet dataframe! **HURRRRRAAAAAAAAYY**

In [30]:
final_df.to_csv(os.path.join(datapath, "Merged UN Data.csv"))

BELOW HERE IS JUST THE PLAYGROUND TO TRY SOME STUFF OUT -> The dataset should be done in the top part

In [5]:
# open my baby again 
df = pd.read_csv(os.path.join(datapath, "Merged UN Data.csv"))

C:\Users\Lenny\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df[3550:3600]
## 3557 drop
dropping_cols = np.arange(3557, df.shape[0])
for i in dropping_cols: 
    df = df.drop(i)
df.shape

In [ ]:
## VALIDATE BEER AND WINE CONSUMPTION
vege = 'vegetable-consumption-per-capita.csv'
wine = 'wine-consumption-per-person.csv'
beer = 'beer-consumption-per-person.csv'


test_df = pd.DataFrame(data = np.zeros( (500, len(interested_countries)) ), 
                        index = np.arange(500),
                        columns = [ country for country in interested_countries])
# save indices as strings to get meaningful names
test_df.index = test_df.index.map(str)


# paste the values into the final_df from each single list
listies = [vege, wine, beer]
nextListIndex = 0

for lists in listies:

    ## Read the file and get the all countries along with their reported years
    print("{} list out of {}, Index: {}, Name: {}".format(type4Cols.index(lists), len(type4Cols), nextListIndex, lists))
    df = pd.read_csv(os.path.join(datapath, str(lists) ) , encoding='latin-1')

    # get the col names, unique countries and unique years
    columns = list(df.columns)
    countries = list(set(df["Entity"]))
    years = list(set(df['Year']))

    # get a list of all the index/row names
    indexNamesArr = test_df.index.values

    ## groupby countries and then years accordingly
    #df.groupby(["Entity", 'Year'])

    firstListRun = False # flag for renaming the indices

    # loop thru every country in the list
    for country in countries:

        ## check if country is in our masterlist
        if country not in interested_countries: 
            # skip this item
            print("\tCountry: {} not in list - but we skip it.".format(country))
            continue

        # take a dataframe for one country at a time
        country_df = df[df['Entity'] == country]

        # loop thru every year within that country - assuming the years are in the same order for every country
        for year in years:

            # rename the indices only if it is the very first run for the country
            if not firstListRun:
                indexName = str(columns[-1]) + ' in ' + str(year)
                indexNamesArr[nextListIndex] = indexName
                nextListIndex += 1
                if nextListIndex % 20 == 0: 
                    print("\t\t" + str(indexName))

            ## get the proper value and fill empty ones, if not available, fill it with 0.000
            # note: .sum() is only having one element anyway, just done to get the value as a float not an array
            value = country_df[country_df["Year"] == year][columns[-1]].sum() if not country_df[country_df["Year"] == year][columns[-1]].empty else 0.000
            # get the name of the row
            idxName = str(columns[-1]) + ' in ' + str(year)

            #print("Country: {} found in the dataset at spot: {}".format(country, interested_countries.index(country)))

            # assign the value in the merged df with the value 
            test_df.iat[list(final_df.index.values).index(idxName), interested_countries.index(country)] = value # .iat[row, col]

        ## get the proper index for the next list to begin with   
        firstListRun = True
        # print progress
        if countries.index(country) % 60 == 0:
            print("\tWorking on country: {} out of {}".format(countries.index(country), len(countries)))


In [ ]:
test_df

In [ ]:
df.groupby('Entity')['Wine Consumption'].sum().sort_values().tail(5)

In [ ]:
df[df['Entity'] == 'Germany']['Wine Consumption'].plot.hist(bins=20)